In [ ]:
import numpy   as np 
x = np.loadtxt("data/train_x.csv", delimiter=",") # load from text 
y = np.loadtxt("data/train_y.csv", delimiter=",") 
x = x.reshape(-1, 64, 64) # reshape 
y = y.reshape(-1, 1)
np.save("data/"+"X_train",x)
np.save("data/"+"y_train",y)
print("X.shape",x.shape, "Y.shape",y.shape)

In [ ]:
import numpy as np
x = np.load("data/X_train.npy")
y = np.load("data/y_train.npy")

In [ ]:
#preprocessing
#using 1 hot encoding encode the 40 unique classes of y 
decode = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 20, 21, 24, 25, 27, 28, 30, 32, 35, 36, 40, 42, 45, 48, 49, 54, 56, 63, 64, 72, 81]
encodedY = np.zeros((50000,40))
for i in range(0,49999):
    index = decode.index(y[i])
    encodedY[i][index] = 1

#flat out X from(64,64) to 4096
#and normalize the input
flatX = np.zeros((50000,4096))
for i in range(0,49999):
    flatX[i] = x[i].flatten()
    #normalize input data
    mean = np.mean(flatX[i])
    flatX[i] = (flatX[i]- mean)/128.0
    
    
#shuffle    
np.random.shuffle(flatX)
np.random.shuffle(encodedY)

#take 90% training set and 10% validation set
train_X = flatX[0:45000]
Valid_X = flatX[45000:50000]
train_Y = encodedY[0:45000]
Valid_X = encodedY[45000:50000]

    

In [ ]:
import numpy as np
from numpy import exp, array, random, dot
from random import shuffle

class FeedForwardNeuralNetwork():
    #Nlayer must be >=2
    def __init__(self,NLayer,Nneuron,ranSeed,Ninput,Noutput,bias):
        #######################################################################################################
        #specify the number of hidden layers , number of neurons needed per layer, number of weights per neuron
        #ranSeed specify the random seed used in initializing weights
        #activation specified what activation function is to be used, pass in 0 to use default sigmoid function
        #######################################################################################################
        np.random.seed(ranSeed)
        self.bias = [bias]*NLayer
        self.Nneuron = Nneuron
        self.Ninput = Ninput
        self.Noutput = Noutput
        self.NLayer = NLayer
        #######################################################################################################
        #input weights, hidden weights and output weights
        self.InputWeight = np.random.uniform(0,1,(Nneuron,Ninput))
        self.HiddenWeight = np.random.uniform(0,1,(NLayer-1,Nneuron,Nneuron))
        self.OutputWeight = np.random.uniform(0,1,(Nneuron,Noutput))
        ######################################################################################################
    
    #####################################################################
    #allows users to manually set all the weights
    def set_Weights(self,InputWeights,OutputWeights,HiddenWeights):
        self.OutputWeight = OutputWeights
        self.InputWeight = InputWeights
        self.HiddenWeight = HiddenWeights
        
    #####################################################################
    #default activation function, normalize input into 0 and 1
    def sigmoid(self, x):
        return 1 / (1 + exp(-x))
    
    #####################################################################
    #The derivative of the Sigmoid function.
    #This is the gradient of the Sigmoid curve.
    def sigmoid_derivative(self, sigmoid):
        return (sigmoid) * (1 - (sigmoid))
    
    #####################################################################
    def softmax(self,inputs):
        f_i = inputs
        log_c = -np.max(f_i)  
        sum_j = np.sum(np.exp(f_i)+log_c)
        out = np.exp(f_i) / sum_j
        return out
    
    #####################################################################
    def cross_entropy_loss(self,px,y):
        return (-y*(np.log(px)))

    ####################################################################
    #derivative of softmax function
    def softmax_derivative(self,target,prediction,O):
        '''
        input:
        target: one hot encoded target output
        prediction: one hot encoded predictions
        O: inputs from hidden layers
        
        output:
        dW: same shape as outputWeight (Nneuron,Noutput)
        '''
        #find the target class
        for i in range(0,len(target)):
            if(target[i] == 1):
                y=i
        num_classes = len(prediction)
        dW = np.zeros((self.Nneuron,self.Noutput))
        for j in range(num_classes):
            dW[:,j] = (prediction[j]-(j == y)) * O[(self.NLayer)-1]
        return dW
        
    #####################################################################
    def softmax_delta(self,prediction,target,dW):
        out = np.zeros(self.Noutput)
        for i in range(0,self.Noutput):
            out[i] = dot(self.OutputWeight[:,i],dW[:,i])
        return out
    
    #####################################################################
    #use to predict output for one hot encoding, largest probability is the output 1, others are 0
    def where(self,inputs):
        return np.where(inputs==inputs.max())
    
    #####################################################################
    #take input a 2d array consists of answers from each sample output
    def Decode(self,inputs):
        Decoded = np.zeros((len(inputs),len(inputs[0])))
        for i in range(len(inputs)):
            x = np.where( inputs[i]==max(inputs[i]))
            Decoded[i][x] = 1
        return Decoded
    
    
    #####################################################################
    #train the neural network
    '''
     Inputs:
      - num_training_example : int 
      - training_inputs: [num_training_example][each_input]
      - training_outputs: [num_training_example][each_output]
      - num_iteration: How many times you use the data to train NN
      - learning rate: recommend 0.001
    '''
    def train(self, num_training_example, training_inputs, training_outputs, num_iteration, learning_rate):
        #use to store outputs
        outputArr = np.zeros((num_training_example,self.Noutput))
        
        for iteration in xrange(num_iteration):
            print("current round is" + " "+ str(iteration))
            for i in range(0,num_training_example):
                #------------------------------------------------
                # delta is the error correction for all neurons in NN
                # delta values are used to backpropagate
                delta = np.zeros((self.NLayer,self.Nneuron))
                deltaOutput = np.zeros(self.Noutput)
                #---------------------------------------------
                # Forward pass
                O,prediction = self.forward(training_inputs[i])
                #---------------------------------------------
                #compute corrections
                # first calculate correction in the output layer
                #----------------------------------------------------
                #update weights for output layer
                Woutput_Update = self.softmax_derivative(training_outputs[i],prediction,O)
                deltaOutput = self.softmax_delta(prediction,training_outputs[i],Woutput_Update)
                #########################################################
                for z in range(0,self.Nneuron):
                    for k in range(0,self.Noutput):
                        self.OutputWeight[z][k] = self.OutputWeight[z][k] + (learning_rate*Woutput_Update[z][k])
                #---------------------------------------------------  
                for z in range(0,self.Nneuron):
                    #calculate the delta values for the last hidden layer
                    delta[self.NLayer-2][z] = self.sigmoid_derivative(O[self.NLayer-1][z])*dot(deltaOutput,self.OutputWeight[z])
                
                #And update weights for bias in output layer
                self.bias[self.NLayer-1] = self.bias[self.NLayer-1] + (learning_rate*Woutput_Update[z][0])
                #update bias for hidden layers and input layer
                for z in range(0,self.NLayer-1):
                    self.bias[z] = self.bias[z] + (learning_rate*Woutput_Update[z][0])        
                #--------------------------------------
                #calculate the backprop for hidden layer
                #j is counting the current layer
                #z is counting the neuron position
                #propagate delta values
                for j in range(self.NLayer-2,-1,-1):
                    for z in range(0,self.Nneuron):
                        #multiple the derivative with all the connected neuron's delta            
                        #Sumdelta will compute all the propagated deltas
                        delta[j][z] = self.sigmoid_derivative(O[j][z])*(self.SumDelta(delta,j,z))
                        #update the weights now
                        for k in range(0,self.Nneuron):
                            self.HiddenWeight[j][z][k] = self.HiddenWeight[j][z][k] + (learning_rate*delta[j][z]*
                            O[j][k])
                #---------------------------------------
        
    #compute the propagated delta of a neuron
    def SumDelta(self,delta,Layer,position):
        output = dot(delta[Layer+1],self.HiddenWeight[Layer][position])
        return output
                
    # pass input through all the layers until the output layer
    def forward(self, inputs):
        '''
        input: 
        inputs from the input layer(shape: self.Ninput)
        output: 
        O: intermediate output from all the hidden layers (shape: (self.NLayer,self.Nneuron))
        Prediction : values in output layer after softmax activation (shape: (self.Noutput,)
        '''
        #O are results within NN and output is obtained from the output layer
        #encoding output layer with multi-class domains, each output neuron represents a discrete output value
        O = np.zeros((self.NLayer,self.Nneuron))
        prediction = np.zeros(self.Noutput)
        ########################################################
        #first pass from input layer to hidden layer
        current = np.zeros(self.Nneuron)
        for i in range(0,self.Nneuron):
            current[i] = self.sigmoid(dot(inputs,self.InputWeight[i])+self.bias[0])
        O[0] = np.copy(current)
        ########################################################
        #passing through hidden layers
        #use input from last layer as input
        for i in range(0,self.NLayer-1):
            for j in range(0,self.Nneuron):
                current[j] = self.sigmoid(dot(O[i],self.HiddenWeight[i][j])+self.bias[i])
            O[i+1] = np.copy(current)
        #now from hidden layer to output layer
        #the weight for the last hidden layer is the weight for the output layer
        #---------------------------------------------------------------
        Out = []
        for i in range(0,self.Noutput):
            Out.append(dot(current,self.OutputWeight[:,i])+self.bias[self.NLayer-1])
        #----------------------------------------------------------------
        prediction = np.copy(self.softmax(Out))
        return (O,prediction)
        
    ##########################################################    
    #a simple function to check the validation accuracy    
    def validate(self, num_validate_example, validating_inputs, validating_outputs):
        count = 0
        for i in range(0,num_validate_example):
            O,prediction = self.forward(validating_inputs[i])
            index = self.where(prediction)
            if( index == self.where(validating_outputs[i])):
                count = count + 1
        print("accuracy is" + str(count*1.0/num_validate_example))
        

In [ ]:
'''
Run code from here
'''
if __name__ == "__main__":

    #Intialise a feedforward neural network
    #def __init__(self,NLayer,Nneuron,ranSeed,Ninput,Noutput,bias):
    neural_network = FeedForwardNeuralNetwork(4,200,1,4096,40,1)
    #def train(self, num_training_example, training_inputs, training_outputs, num_iteration, learning_rate):
    neural_network.train(45000,train_X,train_Y,100,0.001)
    #def validate(self, num_validate_example, validating_inputs, validating_outputs):
    neural_network.validate(5000,valid_X,valid_Y)
    